## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [3]:
import warnings
warnings.filterwarnings("ignore")

!pip install pyconll

In [4]:
import pyconll

In [5]:
!mkdir datasets

Џ®¤Ї ЇЄ  Ё«Ё д ©« datasets г¦Ґ бгйҐбвўгҐв.


In [6]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [7]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu.txt')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu.txt')

In [8]:
def createTaggerList(data):
    outData = []
    for sent in data:
        outTagger = []
        for token in sent:
            outTagger.append((token.form, token.upos, ))

        outData.append(outTagger)
    return outData

train_data = createTaggerList(full_train)
test_data = createTaggerList(full_test)

In [10]:
import nltk
nltk.download('tagsets')
nltk.download('brown')
nltk.download('names')
nltk.download('universal_tagset')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

from nltk.corpus import brown
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger

tags = [tag for (word, tag) in
brown.tagged_words(categories='news')]
nltk.FreqDist(tags)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

#### UnigramTagger

In [ ]:
unigram_tagger = UnigramTagger(train_data)
print(unigram_tagger.tag(full_test[0].text.split()))
print(unigram_tagger.evaluate(test_data))

#### BigramTagger

In [ ]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
print(bigram_tagger.tag(full_test[0].text.split()))
print(bigram_tagger.evaluate(test_data))

#### TrigramTagger

In [ ]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
print(trigram_tagger.tag(full_test[0].text.split()))
print(trigram_tagger.evaluate(test_data))

#### Комбинация тэггеров

In [ ]:
from nltk.tag import TrigramTagger

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

tag = backoff_tagger(train_data,
                     [ BigramTagger, UnigramTagger, TrigramTagger],
                     backoff = DefaultTagger('NN'))

tag.evaluate(test_data)

tag = backoff_tagger(train_data,
                     [ TrigramTagger, BigramTagger, UnigramTagger],
                     backoff = DefaultTagger('NN'))

tag.evaluate(test_data)

In [ ]:
train_tok = []
train_label = []
for sent in train_data[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
test_tok = []
test_label = []
for sent in test_data[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

test_enc_labels = le.transform(test_label)
le.classes_

In [ ]:
hvectorizer = HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100)
hvectorizer_word = HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=100)
cvectorizer = CountVectorizer(ngram_range=(1,2), analyzer="word")

X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)
X_train.shape

X_train_1 = hvectorizer_word.fit_transform(train_tok)
X_test_1 = hvectorizer_word.transform(test_tok)
X_train_1.shape

X_train_2 = cvectorizer.fit_transform(train_tok)
X_test_2 = cvectorizer.transform(test_tok)
X_train_2.shape

In [ ]:
def lr_fit(X_train, X_test):
    lr = LogisticRegression(random_state=0)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    return accuracy_score(test_enc_labels, pred)

print(f"hvectorizer accur:{lr_fit(X_train, X_test)}")
print(f"hvectorizer_word accur:{lr_fit(X_train_1, X_test_1)}")
print(f"cvectorizer accur:{lr_fit(X_train_2, X_test_2)}")

#### Вывод задача №1
1. UnigramTagger BigramTagger TrigramTagger Сразу показали хорошие результаты (видимо из за высокого количества пр). BigramTagger немного вышел в лидеры.
Комбинация тэггеров не дала существенного прироста, даже при различном замешивании
Скор при HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100) = 0.686
Скор при HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=100) = 0.287
Скор при CountVectorizer(ngram_range=(1,2), analyzer="word") = 0.
Каунтвекторайзер(на словах) незначительно уступает Хешвекторайзеру(на буквах). Хешвекторайзер на словах хуже чем на символьном.


#### Создание тэггера имен (NER)

In [ ]:
nltk.download('punkt')

from nltk.tag import SequentialBackoffTagger
from nltk.corpus import names

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in names.words()])

    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
             return 'NNP'
        else:
             return None

nt = NamesTagger()
print(nt.tag(['Katya']))
print(nt.tag(['Adam']))
print(nt.tag(['Window']))

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
nltk.download('maxent_ne_chunker')
nltk.download('words')

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')

#nltk.pos_tag(nltk.word_tokenize(document))
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

In [ ]:
!pip -q install spacy
!python -m spacy download en
!python -m spacy download en_core_web_sm

!pip install -U spacy
!python -m spacy info
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [11]:

# !python -m venv env
# #.\env\Scripts\activate.bat
# !pip install deeppavlov
# !python -m deeppavlov install squad_bert

#!python -m deeppavlov install ner_ontonotes
import deeppavlov
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

AttributeError: 'Struct' object has no attribute 'get'

In [ ]:
from razdel import tokenize

In [ ]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [ ]:
import pandas as pd

In [ ]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [ ]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_x.apply(len).max(axis=0)
# char level
#train_x = train_x.apply(lambda x: ' '.join(list(x)))
#valid_x = valid_x.apply(lambda x: ' '.join(list(x)))

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [ ]:
len(vectorize_layer.get_vocabulary())

In [ ]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)

        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)

        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
mmodel = modelNER()

In [ ]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)